In [2]:
import polars as pl
import pandas as pd

c:\Users\Mert\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
lazy_df = pl.scan_csv(r"C:\Users\Mert\OneDrive\Desktop\Courses\Financial Big Data\data\nasdaq_exteral_data.csv")

In [4]:
lazy_df.columns

C:\Users\Mert\AppData\Local\Temp\ipykernel_26060\4236129083.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lazy_df.columns


['Unnamed: 0',
 'Date',
 'Article_title',
 'Stock_symbol',
 'Url',
 'Publisher',
 'Author',
 'Article',
 'Lsa_summary',
 'Luhn_summary',
 'Textrank_summary',
 'Lexrank_summary']

In [5]:
filtered_df = (
    lazy_df
    .select(["Date", "Stock_symbol", "Article_title"])  # Select specific columns
    .filter(pl.col("Article_title").is_not_null())  # Filter null values
    .filter(pl.col("Stock_symbol") == "AAPL")  # Filter Stock_symbol == AAPL 
    .with_columns(pl.col("Date").str.to_datetime("%Y-%m-%d %H:%M:%S UTC"))  # Convert Date to proper datetime format
)

In [ ]:
apple_df = (pd.DataFrame(filtered_df.collect())
            .dropna()
            .drop_duplicates()
            .rename({0 : "Date",1 : "Stock_Symbol",2 : "Article_title"}, axis = 1)
            )

In [10]:
apple_df.to_csv("../data/apple_news.csv")